In [47]:
import pandas as pd
import cPickle as pickle

import numpy as np
import geopandas as gpd
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.mixture import GaussianMixture

from shapely.geometry import Point, MultiPoint

## Load

In [2]:
userLoc = pd.read_pickle("./USI-Impact-L-Train-Shutdown/data/userLoc.p")
# userLoc["MrNaassir"][1].x; userLoc["MrNaassir"][1].y

## Format

In [9]:
geo = {}
for user, pList in userLoc.iteritems():
    X = []
    for p in pList:
        X.append([p.y, p.x])
    geo[user] = X
# X = geo["MrNaassir"]

## Function: Gaussian Mixture Clustering

In [150]:
def homeLoc(user):
    X = geo[user]
    # At least sent 4 tweets, else return centroid directly. 
    if len(X) < 4:
        return MultiPoint(X).centroid
    
    maxK = 10
    silhouScore = 0
    # At least ONE cluster
    finalClusters = 1 

    for n_clusters in range(2, min(maxK, len(X)-1)):
        #run the clustering
        gm=GaussianMixture(n_components=n_clusters, random_state=324)
        #assign labels of clusters
        cluster_labels=gm.fit(X).predict(X)
        #calculate average for each cluster
        if len(set(cluster_labels)) != 1:
            silhouette_avg = silhouette_score(X, cluster_labels)
            if silhouette_avg > silhouScore:
                silhouScore = silhouette_avg
                finalClusters = n_clusters


    gm=GaussianMixture(n_components=finalClusters, random_state=324)
    res1=gm.fit(X).predict(X)
    target_cluster = pd.Series(res1).value_counts().argmax()
    label = res1 == target_cluster

    X = pd.Series(X).loc[label]
    XX = list(X)
    return MultiPoint(XX).centroid

# homeLoc("MrNaassir")
# homeLoc("lancewhite")

## Run 

In [151]:
len(geo.keys())

31337

In [162]:
home = {}
for i, user in enumerate(geo.keys()):
    #print user
    print "\r", 100.0*(i+1)/len(geo.keys()), "%",
    home[user] = homeLoc(user)

100.0 %


## Output

In [163]:
with open('./USI-Impact-L-Train-Shutdown/data/userLoc_GaussianMixture.p', 'wb') as f:
    pickle.dump(home, f)